In [ ]:
# Task 1: Build container

export MonolithIdentifier=fancytest

git clone https://github.com/googlecodelabs/monolith-to-microservices.git
    
cd ~/monolith-to-microservices
./setup.sh
nvm install --lts

cd ~/monolith-to-microservices/monolith
npm start

gcloud services enable cloudbuild.googleapis.com
gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/$MonolithIdentifier:1.0.0 .
    
